   
## [mlcourse.ai](https://mlcourse.ai) - Curso abierto de *Machine Learning* 

Autor: [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/). Traducido y editado por [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). Este material está sujeto a los términos y condiciones de la licencia [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Se permite su uso libre para cualquier fin no comercial.

# <center>Tema 9. Análisis de series temporales en Python</center>
## <center>Parte 2. Predecir el futuro con Facebook Prophet</center>

La predicción de series temporales encuentra una amplia aplicación en el análisis de datos. Estas son sólo algunas de las predicciones concebibles de tendencias futuras que podrían ser útiles:
- El número de servidores que necesitará un servicio en línea el año que viene.
- La demanda de un producto de alimentación en un supermercado en un día determinado.
- El precio de cierre de mañana de un activo financiero negociable.

Otro ejemplo: podemos predecir el rendimiento de un equipo y utilizarlo como referencia: primero para fijar los objetivos del equipo y después para medir el rendimiento real del equipo en relación con la referencia.

Existen bastantes métodos diferentes para predecir tendencias futuras, por ejemplo, [ARIMA](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average), [ARCH](https://en.wikipedia.org/wiki/Autoregressive_conditional_heteroskedasticity), [modelos regresivos](https://en.wikipedia.org/wiki/Autoregressive_model), [redes neuronales](https://medium.com/machine-learning-world/neural-networks-for-algorithmic-trading-1-2-correct-time-series-forecasting-backtesting-9776bfd9e589).

En este artículo, examinaremos [Prophet](https://facebook.github.io/prophet/), una biblioteca para la previsión de series temporales lanzada por Facebook y de código abierto el 23 de febrero de 2017. También la probaremos en el problema de predecir el número diario de publicaciones en Medium.


## Índice

1. Introducción
2. El modelo de predicción de Prophet
3. Práctica con Prophet
    * 3.1 Instalación en Python
    * 3.2 Conjunto de datos
    * 3.3 Análisis visual exploratorio
    * 3.4 Realización de una predicción
    * 3.5 Evaluación de la calidad de la predicción
    * 3.6 Visualización
4. Transformación Box-Cox
5. Resumen
6. Referencias

## 1. Introducción

Según el [artículo](https://research.fb.com/prophet-forecasting-at-scale/) de Facebook Research, Prophet se desarrolló inicialmente con el propósito de crear predicciones empresariales de alta calidad. Esta biblioteca trata de abordar las siguientes dificultades comunes a muchas series temporales empresariales:
- Efectos estacionales causados por el comportamiento humano: ciclos semanales, mensuales y anuales, caídas y picos en días festivos.
- Cambios de tendencia debidos a nuevos productos y acontecimientos del mercado.
- Valores atípicos.

Los autores afirman que, incluso con la configuración por defecto, en muchos casos su biblioteca produce predicciones tan precisas como las de analistas experimentados.

Además, Prophet dispone de una serie de personalizaciones intuitivas y fácilmente interpretables que permiten mejorar gradualmente la calidad del modelo de predicción. Lo que es especialmente importante, estos parámetros son bastante comprensibles incluso para los no expertos en análisis de series temporales, que es un campo de la ciencia de datos que requiere cierta habilidad y experiencia.

Por cierto, el artículo original se titula "Forecasting at Scale", pero no trata de la escala en el sentido "habitual", es decir, abordar los problemas computacionales y de infraestructura de un gran número de programas en funcionamiento. Según los autores, Prophet debe escalar bien en las 3 áreas siguientes:
- Accesibilidad a un amplio público de analistas, posiblemente sin profundos conocimientos en series temporales.
- Aplicabilidad a una amplia gama de problemas de predicción distintos.
- Estimación automatizada del rendimiento de un gran número de predicciones, incluida la señalización de problemas potenciales para su posterior inspección por parte del analista.

## 2. El modelo de predicción de Prophet

Veamos ahora con más detalle cómo funciona Prophet. En su esencia, esta biblioteca utiliza el [modelo de regresión aditiva](https://en.wikipedia.org/wiki/Additive_model) $y(t)$ que comprende las siguientes componentes:

$$y(t) = g(t) + s(t) + h(t) + \epsilon_{t},$$

donde:
* La tendencia $g(t)$ modela los cambios no periódicos.
* La estacionalidad $s(t)$ representa los cambios periódicos.
* La componente de vacaciones $h(t)$ aporta información sobre vacaciones y eventos.

A continuación, consideraremos algunas propiedades importantes de estas componentes del modelo.

### Tendencia

La biblioteca Prophet implementa dos posibles modelos de tendencia para $g(t)$.

El primero se denomina *Crecimiento no lineal, saturante*. Se representa en forma de [modelo de crecimiento logístico](https://en.wikipedia.org/wiki/Logistic_function):


$$g(t) = \frac{C}{1+e^{-k(t - m)}},$$

donde:
* $C$ es la capacidad de carga (es decir, el valor máximo de la curva).
* $k$ es la tasa de crecimiento (que representa "la inclinación" de la curva).
* $m$ es un parámetro de compensación.

Esta ecuación logística permite modelizar el crecimiento no lineal con saturación, es decir, cuando la tasa de crecimiento de un valor disminuye con su crecimiento. Uno de los ejemplos típicos sería representar el crecimiento de la audiencia de una aplicación o una página web.

En realidad, $C$ y $k$ no son necesariamente constantes y pueden variar con el tiempo. Prophet admite tanto el ajuste automático como el manual de su variabilidad. La biblioteca puede elegir por sí misma los puntos óptimos de los cambios de tendencia ajustándose a los datos históricos suministrados. 

Además, Prophet permite a los analistas fijar manualmente los puntos de cambio de los valores de la tasa de crecimiento y la capacidad en distintos momentos. Por ejemplo, los analistas pueden tener información sobre fechas de lanzamientos pasados que influyeron de forma destacada en algunos indicadores clave del producto.

El segundo modelo de tendencia es un simple *Modelo lineal por partes* con una tasa de crecimiento constante. Es el más adecuado para problemas sin crecimiento saturado.

### Estacionalidad

La componente estacional $s(t)$ proporciona un modelo flexible de los cambios periódicos debidos a la estacionalidad semanal y anual.

Los datos estacionales semanales se modelan con variables ficticias. Se añaden seis nuevas variables: `monday`, `tuesday`, `wednesday`, `thursday`, `friday`, `saturday`, que toman valores 0 ó 1 según el día de la semana. La característica "domingo" no se añade porque sería una combinación lineal de los demás días de la semana, y este hecho tendría un efecto adverso en el modelo.

El modelo de estacionalidad anual de Prophet se basa en series de Fourier.

Desde la [versión 0.2](https://github.com/facebook/prophet) también puede utilizar *sub-series temporales diarias* y realizar *previsiones subdiarias*, así como emplear la nueva función de *estacionalidad diaria*.

### Días festivos y eventos

La componente $h(t)$ representa los días anormales previsibles del año, incluidos los de calendario irregular, por ejemplo, los Black Fridays.

Para utilizar esta función, el analista debe proporcionar una lista personalizada de eventos.

### Error

El término de error $\epsilon(t)$ representa información no reflejada en el modelo. Normalmente se modela como ruido distribuido normalmente.

### Prophet Benchmarking

Para una descripción detallada del modelo y los algoritmos de Prophet, consulte el artículo ["Forecasting at scale"](https://peerj.com/preprints/3190/) de Sean J. Taylor y Benjamin Letham.

Los autores también compararon su biblioteca con otros métodos de predicción de series temporales. Utilizaron [Mean Absolute Percentage Error (MAPE)](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error) como medida de la precisión de la predicción. En esta investigación, Prophet ha mostrado un error de predicción sustancialmente menor que los otros modelos.

<img src="img/topic9_benchmarking_prophet.png" />

Veamos con más detalle cómo se ha medido la calidad de las predicciones en el artículo. Para ello, necesitaremos la fórmula del error porcentual absoluto medio (*MAPE*).

Sea $y_{i}$ el *valor real (histórico)* y $\hat{y}_{i}$ el *valor predicho* por nuestro modelo.

A continuación, $e_{i} = y_{i} - \hat{y}_{i}$ es el *error de predicción* y $p_{i} = \frac{e_{i}}{y_{i}}$ es el *error relativo de predicción*.

Definimos

$$MAPE = mean\big(\left |p_{i}\right |\big)$$

*MAPE* se utiliza ampliamente como medida de la precisión de la predicción porque expresa el error como porcentaje y, por tanto, puede utilizarse en evaluaciones de modelos en diferentes conjuntos de datos.

Además, al evaluar un algoritmo de predicción, puede resultar útil calcular [*MAE* (Error Medio Absoluto)](https://en.wikipedia.org/wiki/Mean_absolute_error) para tener una idea de los errores en números absolutos. Utilizando los componentes definidos anteriormente, su ecuación será

$$MAE = mean\big(\left |e_{i}\right |\big)$$

Unas palabras sobre los algoritmos con los que se comparó Prophet. La mayoría de ellos son bastante sencillos y suelen utilizarse como referencia para otros modelos:
* `naive` (ingenuo) es un enfoque de predicción simplista en el que predecimos todos los valores futuros basándonos únicamente en la observación en el último momento disponible.
* El modelo `snaive` (ingenuo estacional) realiza predicciones constantes teniendo en cuenta la información sobre la estacionalidad. Por ejemplo, en el caso de datos estacionales semanales, para cada lunes futuro se predeciría el valor del último lunes, y para todos los martes futuros se utilizaría el valor del último martes, y así sucesivamente.
* `mean` utiliza el valor medio de los datos como predicción.
* `arima` significa *Media Móvil Autorregresiva Integrada*, véase [Wikipedia](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average) para más detalles.
* `ets` significa *Suavizado exponencial*, véase [Wikipedia](https://en.wikipedia.org/wiki/Exponential_smoothing) para más información.

## 3. Práctica con Prophet

### 3.1 Instalación en Python

En primer lugar, necesitas instalar la librería. Prophet está disponible para Python y R. La elección dependerá de tus preferencias personales y de los requisitos del proyecto. En este *notebook* utilizaremos Python.

En Python puedes instalar Prophet usando PyPI:

```
$ pip install fbprophet
```

En R puede encontrar el paquete CRAN correspondiente. Consulta la [documentación](https://facebookincubator.github.io/prophet/docs/installation.html) para más detalles.

Importemos los módulos que necesitaremos e inicialicemos nuestro entorno:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

%matplotlib inline

### 3.2 *Dataset*

Vamos a predecir el número de posts diarios publicados en [Medium](https://medium.com/).

Primero, cargamos nuestro *dataset*.

In [ ]:
df = pd.read_csv('data/medium_posts.csv', sep='\t')
display(df.shape, df.head())

A continuación, omitimos todas las columnas excepto `published` y `url`. La primera corresponde a la dimensión temporal, mientras que la segunda identifica de forma exclusiva una entrada por su URL. De este modo, eliminamos los posibles duplicados y los valores que faltan en los datos:

In [ ]:
df = df[['published', 'url']].dropna().drop_duplicates()
display(df.shape, df.head())

A continuación, tenemos que convertir `published` al formato datetime porque por defecto `pandas` trata este campo como string-valued.

In [ ]:
df['published'] = pd.to_datetime(df['published'])

Ordenemos el marco de datos por tiempo y echemos un vistazo a lo que tenemos:

In [ ]:
df.sort_values(by=['published']).head(3)

La fecha de publicación de Medium fue el 15 de agosto de 2012. Pero, como se puede ver en los datos anteriores, hay al menos varias filas con fechas de publicación muy anteriores. De alguna manera han aparecido en nuestro conjunto de datos, pero difícilmente son legítimas. Nos limitaremos a recortar nuestra serie temporal para conservar únicamente las filas que se sitúan en el periodo comprendido entre el 15 de agosto de 2012 y el 25 de junio de 2017:

In [ ]:
df = df[(df['published'] > '2012-08-15') & (df['published'] < '2017-06-26')].sort_values(by=['published'])
df.head(3)

In [ ]:
df.tail(n=3)

Como vamos a predecir el número de entradas publicadas, agregaremos y contaremos las entradas únicas en cada momento. Denominaremos a la nueva columna correspondiente `posts`:

In [ ]:
aggr_df = df.groupby('published')[['url']].count()
aggr_df.columns = ['posts']
display(aggr_df.shape, aggr_df.head())

En esta práctica, nos interesa el número de mensajes **al día**. Pero en este momento todos nuestros datos están divididos en intervalos de tiempo irregulares que son inferiores a un día. Esto se denomina *serie temporal subdiaria*. Para verlo, imprimamos las 3 primeras filas:

In [ ]:
aggr_df.head(n=3)

Para solucionarlo, tenemos que agregar los recuentos de entradas por "intervalos" de un tamaño de fecha. En el análisis de series temporales, este proceso se denomina *remuestreo*. Y si *reducimos* la tasa de muestreo de los datos se suele llamar *downsampling*.

Por suerte, `pandas` tiene una funcionalidad incorporada para esta tarea. Vamos a remuestrear nuestro índice de tiempo a intervalos de 1 día:

In [ ]:
daily_df = aggr_df.resample('D').apply(sum)
daily_df.head(n=3)

### 3.3 Análisis visual exploratorio

Como siempre, puede resultar útil e instructivo observar una representación gráfica de los datos.

Crearemos un gráfico de series temporales para todo el intervalo de tiempo. La visualización de los datos durante un periodo de tiempo tan largo puede dar pistas sobre la estacionalidad y las desviaciones anormales llamativas.

En primer lugar, importamos e inicializamos la librería `Plotly`, que permite crear bonitos gráficos interactivos:

In [ ]:
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

# Initialize plotly
init_notebook_mode(connected=True)

También definimos una función de ayuda, que trazará nuestros *datasets* a lo largo del *notebook*:

In [ ]:
def plotly_df(df, title=''):
    """Visualize all the dataframe columns as line plots."""
    common_kw = dict(x=df.index, mode='lines')
    data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title)
    fig = dict(data=data, layout=layout)
    iplot(fig, show_link=False)

Intentemos trazar nuestro conjunto de datos *tal cual*:

In [ ]:
plotly_df(daily_df, title='Posts on Medium (daily)')

Los datos de alta frecuencia pueden ser bastante difíciles de analizar. Incluso con la posibilidad de hacer zoom que ofrece `Plotly`, es difícil deducir algo significativo de este gráfico, además de la prominente tendencia al alza y la aceleración.

Para reducir el ruido, volveremos a muestrear los recuentos de mensajes en intervalos semanales. Además del *binning*, otras posibles técnicas de reducción del ruido son el [Suavizado de medias móviles](https://en.wikipedia.org/wiki/Moving_average) y el [Suavizado exponencial](https://en.wikipedia.org/wiki/Exponential_smoothing), entre otras.

Guardamos nuestro *dataset* en una variable separada porque en esta práctica sólo trabajaremos con series diarias:

In [ ]:
weekly_df = daily_df.resample('W').apply(sum)

Finalmente, visualizamos el resultado:

In [ ]:
plotly_df(weekly_df, title='Posts on Medium (weekly)')

Este gráfico con muestreo reducido resulta algo mejor para la percepción de un analista.

Una de las funciones más útiles que ofrece `Plotly` es la posibilidad de sumergirse rápidamente en distintos periodos de la línea temporal para comprender mejor los datos y encontrar pistas visuales sobre posibles tendencias, efectos periódicos e irregulares. 

Por ejemplo, hacer zoom en un par de años consecutivos nos muestra puntos temporales correspondientes a las fiestas navideñas, que influyen enormemente en los comportamientos humanos.

Ahora, vamos a omitir los primeros años de observaciones, hasta 2015. En primer lugar, no contribuirán mucho a la calidad de las previsiones en 2017. En segundo lugar, es probable que estos primeros años, con un número muy bajo de publicaciones al día, aumenten el ruido en nuestras predicciones, ya que el modelo se vería obligado a ajustar estos datos históricos anómalos junto con datos más relevantes e indicativos de los últimos años.

In [ ]:
daily_df = daily_df.loc[daily_df.index >= '2015-01-01']
daily_df.head(n=3)

En resumen, del análisis visual se observa que nuestro conjunto de datos es no estacionario y presenta una marcada tendencia creciente. También muestra estacionalidad semanal y anual y un número de días anormales cada año.

### 3.4 Ejecutar una predicción

La API de Prophet es muy similar a la que puedes encontrar en `sklearn`. Primero creamos un modelo, luego llamamos al método `fit`, y, finalmente, hacemos una predicción. La entrada del método `fit` es un `DataFrame` con dos columnas:
* `ds` (*datestamp*) debe ser de tipo `date` o `datetime`.
* `y` es un valor numérico que queremos predecir.

Para empezar, importaremos la librería y silenciaremos los mensajes de diagnóstico sin importancia:

In [ ]:
from prophet import Prophet

import logging
logging.getLogger().setLevel(logging.ERROR)

Convirtamos nuestro *dataset* al formato requerido por Prophet:

In [ ]:
df = daily_df.reset_index()
df.columns = ['ds', 'y']
display(df.head(n=3), df.tail(n=3))

Los autores de la biblioteca suelen aconsejar hacer predicciones basadas en al menos varios meses, e idealmente, más de un año de datos históricos. Por suerte, en nuestro caso tenemos más de un par de años de datos para ajustar el modelo.

Para medir la calidad de nuestra predicción, tenemos que dividir nuestro conjunto de datos en la *parte histórica*, que es la primera y mayor porción de nuestros datos, y la *parte de predicción*, que se situará al final de la línea temporal. Eliminaremos el último mes del conjunto de datos para utilizarlo posteriormente como objetivo de predicción:

In [ ]:
prediction_size = 30
df.ds = df.ds.astype(str).str.split(' ').str[0]
df.ds = pd.to_datetime(df.ds)
train_df = df[:-prediction_size]
display(train_df.head(n=3), train_df.tail(n=3))

Ahora necesitamos crear un nuevo objeto `Prophet`. Aquí podemos pasar los parámetros del modelo en el constructor. Pero en este *notebook* vamos a utilizar los valores por defecto. Luego entrenamos nuestro modelo invocando su método `fit` en nuestro conjunto de datos de entrenamiento:

In [ ]:
m = Prophet()  # Si da el siguiente error: "AttributeError: 'Prophet' object has no attribute 'stan_backend'". Ejecutad en la terminal: pip install --upgrade pystan.
m.fit(train_df);

Usando el método de ayuda `Prophet.make_future_dataframe`, creamos un *dataset* que contendrá todas las fechas del histórico y también se extenderá hacia el futuro para esos 30 días que dejamos fuera antes.

In [ ]:
future = m.make_future_dataframe(periods=prediction_size)
future.tail(n=3)

Predecimos valores con `Prophet` introduciendo las fechas para las que queremos crear una predicción. Si también proporcionamos las fechas históricas (como en nuestro caso), además de la predicción obtendremos un ajuste dentro de la muestra para la historia. Llamemos al método `predict` del modelo con nuestro *dataset* `future` como entrada:

In [ ]:
forecast = m.predict(future)
forecast.tail(n=3)

En el *dataset* resultante puede ver muchas columnas que caracterizan la predicción, incluidos los componentes de tendencia y estacionalidad, así como sus intervalos de confianza. La propia predicción se almacena en la columna `yhat`.

La biblioteca Prophet tiene sus propias herramientas de visualización que nos permiten evaluar rápidamente el resultado.

En primer lugar, existe un método llamado `Prophet.plot` que traza todos los puntos de la predicción:

In [ ]:
m.plot(forecast);

Este gráfico no parece muy informativo. La única conclusión definitiva que podemos sacar aquí es que el modelo trató muchos de los puntos de datos como valores atípicos.

La segunda función `Prophet.plot_components` podría ser mucho más útil en nuestro caso. Nos permite observar por separado las distintas componentes del modelo: tendencia, estacionalidad anual y semanal. Además, si proporciona información sobre días festivos y eventos a su modelo, también se mostrarán en este gráfico.

Vamos a probarlo:

In [ ]:
m.plot_components(forecast);

Como se puede ver en el gráfico de tendencias, Prophet hizo un buen trabajo al encajar el crecimiento acelerado de nuevos posts a finales de 2016. El gráfico de estacionalidad semanal llega a la conclusión de que normalmente hay menos entradas nuevas los sábados y domingos que el resto de días de la semana. En el gráfico de estacionalidad anual se observa un descenso destacado el día de Navidad.

### 3.5 Evaluación de las predicciones

Evaluemos la calidad del algoritmo calculando la métrica del error para los últimos 30 días que hemos predicho. Para ello, necesitaremos las observaciones $y_i$ y los correspondientes valores predichos $\hat{y}_i$.

Vamos a ver en el objeto `forecast` que la biblioteca creado para nosotros:

In [ ]:
print(', '.join(forecast.columns))

Podemos ver que este *dataset* contiene toda la información que necesitamos excepto los valores históricos. Necesitamos unir el objeto `forecast` con los valores reales `y` del conjunto de datos original `df`. Para ello definiremos una función de ayuda que reutilizaremos más adelante:

In [ ]:
def make_comparison_dataframe(historical, forecast):
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))


Apliquemos esta función a nuestra última predicción:

In [ ]:
cmp_df = make_comparison_dataframe(df, forecast)
display(df, forecast, cmp_df.tail(n=3))

También vamos a definir una función de ayuda que utilizaremos para medir la calidad de nuestras predicciones con las medidas de error *MAPE* y *MAE*:

In [ ]:
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    
    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Recall that we held out the values of the last `prediction_size` days
    # in order to predict them and measure the quality of the model. 
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}

Usemos nuestra función:

In [ ]:
for err_name, err_value in calculate_forecast_errors(cmp_df, prediction_size).items():
    print(err_name, err_value)

Como resultado, el error relativo de nuestra predicción (*MAPE*) se sitúa en torno al 22.72%, y por término medio nuestro modelo se equivoca en 70.45 posts (*MAE*).

### 3.6 Visualización

Vamos a crear nuestra propia visualización del modelo construido por Prophet. Comprenderá los valores reales, la predicción y los intervalos de confianza.

En primer lugar, representaremos los datos de un periodo de tiempo más corto para que los puntos de datos sean más fáciles de distinguir. En segundo lugar, mostraremos el rendimiento del modelo sólo para el periodo que hemos predicho, es decir, los últimos 30 días. Parece que con estas dos medidas obtendremos un gráfico más legible.

En tercer lugar, vamos a utilizar `Plotly` para hacer nuestro gráfico interactivo, que es ideal para explorar.

Definiremos una función de ayuda personalizada `show_forecast` y la llamaremos (para más información sobre su funcionamiento, consulta los comentarios del código y la [documentación](https://plot.ly/python/)):

In [ ]:
def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Lower Bound', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color='DarkSlateGrey'))
    upper_bound = create_go('Upper Bound', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color='DarkSlateGrey'),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Forecast', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Actual', 'y', num_values,
                       marker=dict(color="red"))
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='Posts'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

show_forecast(cmp_df, prediction_size, 100, 'New posts on Medium')

A primera vista, la predicción de los valores medios por nuestro modelo parece sensata. El elevado valor de *MAPE* que obtuvimos anteriormente puede explicarse por el hecho de que el modelo no consiguió captar el aumento de la amplitud pico a pico de la estacionalidad débil.

Además, podemos concluir del gráfico anterior que muchos de los valores reales se sitúan fuera del intervalo de confianza. Puede que Prophet no sea adecuado para series temporales con varianza inestable, al menos cuando se utilizan los ajustes por defecto. Intentaremos solucionarlo aplicando una transformación a nuestros datos.

## 4. Transformación Box-Cox

Hasta ahora hemos utilizado Prophet con los parámetros por defecto y los datos originales. Dejaremos solo los parámetros del modelo. Pero, a pesar de ello, aún tenemos margen de mejora. En esta sección, aplicaremos la [transformación Box-Cox](http://onlinestatbook.com/2/transformations/box-cox.html) a nuestra serie original. Veamos a dónde nos lleva.

Unas palabras sobre esta transformación. Se trata de una transformación monótona de datos que puede utilizarse para estabilizar la varianza. Utilizaremos la transformación de Box-Cox de un parámetro, que se define mediante la siguiente expresión:

$$
\begin{equation}
  boxcox^{(\lambda)}(y_{i}) = \begin{cases}
    \frac{\displaystyle y_{i}^{\lambda} - 1}{\displaystyle \lambda} &, \text{if $\lambda \neq 0$}.\\
    ln(y_{i}) &, \text{if $\lambda = 0$}.
  \end{cases}
\end{equation}
$$
:
Necesitaremos implementar la inversa de esta función para poder restaurar la escala original de los datos. Es fácil ver que la inversa se define como:

$$
\begin{equation}
  invboxcox^{(\lambda)}(y_{i}) = \begin{cases}
    e^{\left (\frac{\displaystyle ln(\lambda y_{i} + 1)}{\displaystyle \lambda} \right )} &, \text{if $\lambda \neq 0$}.\\
    e^{y_{i}} &, \text{if $\lambda = 0$}.
  \end{cases}
\end{equation}
$$

La función correspondiente en Python se implementa del siguiente modo:

In [ ]:
def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

En primer lugar, preparamos nuestro conjunto de datos estableciendo su índice:

In [ ]:
train_df2 = train_df.copy().set_index('ds')

A continuación, aplicamos la función `stats.boxcox` de `Scipy`, que aplica la transformación Box-Cox. En nuestro caso devolverá dos valores. El primero es la serie transformada y el segundo es el valor encontrado de $\lambda$ que es óptimo en términos de máxima log-verosimilitud:

In [ ]:
train_df2['y'], lambda_prophet = stats.boxcox(train_df2['y'])
train_df2.reset_index(inplace=True)

In [ ]:
train_df2

Creamos un nuevo modelo `Prophet` y repetimos el ciclo de ajuste-predicción que ya hemos realizado anteriormente:

In [ ]:
m2 = Prophet()
m2.fit(train_df2)
future2 = m2.make_future_dataframe(periods=prediction_size)
forecast2 = m2.predict(future2)

En este punto, tenemos que revertir la transformación Box-Cox con nuestra función inversa y el valor conocido de $\lambda$:

In [ ]:
for column in ['yhat', 'yhat_lower', 'yhat_upper']:
    forecast2[column] = inverse_boxcox(forecast2[column], lambda_prophet)

Aquí reutilizaremos nuestras herramientas para hacer el *dataset* de comparación y calcular los errores:

In [ ]:
cmp_df2 = make_comparison_dataframe(df, forecast2)
for err_name, err_value in calculate_forecast_errors(cmp_df2, prediction_size).items():
    print(err_name, err_value)

Así pues, podemos afirmar sin lugar a dudas que ha aumentado la calidad del modelo.

Por último, vamos a comparar nuestro rendimiento anterior con los últimos resultados. Obsérvese que utilizamos `prediction_size` como tercer parámetro para ampliar el intervalo de predicción:

In [ ]:
show_forecast(cmp_df, prediction_size, 100, 'No transformations')
show_forecast(cmp_df2, prediction_size, 100, 'Box–Cox transformation')

Vemos que la predicción de cambios semanales del segundo gráfico se acerca mucho más a los valores reales actuales.

## 5. Resumen

Hemos echado un vistazo a *Prophet*, una biblioteca de predicción de código abierto orientada específicamente a las series temporales empresariales. También hemos realizado algunas prácticas de predicción de series temporales.

Como hemos visto, la biblioteca Prophet no hace maravillas, y sus predicciones out-of-box no son [ideales](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization). Sigue correspondiendo al científico de datos explorar los resultados de la predicción, ajustar los parámetros del modelo y transformar los datos cuando sea necesario.

Sin embargo, esta biblioteca es fácil de usar y personalizar. La sola posibilidad de tener en cuenta los días anormales que el analista conoce de antemano podría marcar la diferencia en algunos casos.

En definitiva, merece la pena que la biblioteca Prophet forme parte de su caja de herramientas analíticas.

## 6. Referencias

* Repositorio oficial [Prophet](https://github.com/facebookincubator/prophet) en GitHub.
* Documentación oficial [Prophet](https://facebookincubator.github.io/prophet/docs/quick_start.html).
* Sean J. Taylor, Benjamin Letham ["Forecasting at scale"](https://facebookincubator.github.io/prophet/static/prophet_paper_20170113.pdf) - artículo científico en el que se explica el algoritmo que sienta las bases de `Prophet`.
* [Forecasting Website Traffic Using Facebook's Prophet Library](http://pbpython.com/prophet-overview.html) - descripción general de `Prophet` con un ejemplo de previsión del tráfico de un sitio web.
* Rob J. Hyndman, George Athanasopoulos ["Forecasting: principles and practice"](https://www.otexts.org/fpp) - un libro en línea muy bueno sobre predicción de series temporales.